In [1]:
#Extracting libraries:
import sqlite3
import pandas as pd
import numpy as np
import re
import datetime
import os
import requests
from bs4 import BeautifulSoup

In [2]:
#Extracting dirty tables from local-data:
df1 = pd.read_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/df_dirt.csv')

In [3]:
#Display datasets:
display(df1)

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [4]:
#DATA CLEANING

    #eliminating columns that do not add value (id, Unnamed:0,Unnamed: 0_x, Unnamed: 0_y  realTimePosition)
df1.drop(['id'], axis='columns', inplace=True)
df1.drop(['realTimeWorth'], axis='columns', inplace=True)    
df1.drop(['Unnamed: 0'], axis='columns', inplace=True)
df1.drop(['Unnamed: 0_x'], axis='columns', inplace=True)
df1.drop(['Unnamed: 0_y'], axis='columns', inplace=True)
df1.drop(['realTimePosition'], axis='columns', inplace=True)
df1.drop(['lastName'], axis='columns', inplace=True)

    #wrangling worth and worthChange column:
df1.replace({"BUSD": ' '})
df1.worthChange.replace('na', '0.0', regex=True, inplace=True)
df1.worth.replace('[BUSD]+$', '', regex=True, inplace=True)
df1.worthChange.replace('[millions USD]+$', '', regex=True, inplace=True)
df1['worth'] = df1['worth'].astype('float').astype('int')
df1['worthChange'] = df1['worthChange'].astype('float').astype('int')

    #checking the type of values in columns
#print(df['worthChange'].dtype)
#print(df['worth'].dtype)

    #Analyzing worthChange column composition:
#print(df['worthChange'].value_counts())
#print(df['worthChange'].describe())

    #Once the value_counts () and describe () methods have been performed, we can see that the column is composed of 97.59% of 0 values, however we can see that they include very extreme values in minimums and maximums, and since it is a column That refers to the exchange rates I think it is convenient not to eliminate the column for futures and possible analyzes during the project.

    #I rename columns to indicate the units in which the values are expressed and not to contaminate the dataset.

df1 = df1.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})

    #Make sure there are no duplicate items
df1 = df1.drop_duplicates()

    #Wrangling gender column:
    
df1.gender.replace('[M]+$', 'Male', regex=True, inplace=True)
df1.gender.replace('[F]+$', 'Female', regex=True, inplace=True)
df1['gender'] = df1['gender'].astype('object').astype('str')

    #Wrangling age column:
        #Correct info in age column: delete "years old" and change data types
        
df1.age.replace('[years old]+$', '', regex=True, inplace=True)      
df1.age.fillna(str(-2019), inplace=True)
current_year = datetime.datetime.today().year

age = []
for i in df1['age']:
    if len(str(i)) == 4:
        age.append(str(current_year-int(i)))
    else:
         age.append(i)

df1['age'] = age

#print(df_rank_info.position.isnull().sum())   #Only 52 NaN values, not proceed delete column
    #replacement float values with int

df1['position'] = df1['position'].fillna(0)   #replace NaN values with 0.0
df1['position'] = df1['position'].astype('float').astype('int')

    #convert all characters to lowercase in column name

df1 = df1.apply(lambda x: x.astype(str).str.lower())

    #analyzing types of values to see the next cleaning stage
    #we can verify that we have duplicate elements so we must eliminate them but taking care that we eliminate 
    #the correct one and #that it contributes less value to the dataframe.
    
    #changing object types
    
df1['position'] = df1['position'].astype('object').astype('int')

#print(df_rank_info['position'].dtype)
#print(df_rank_info['id'].dtype)

    #I delete the rows that have a value of 0 in the position column, they do not add value to the analysis.
    
df1 = df1.drop(df1[df1['position']==0].index)
    
    #Wrangling Source column:

#df1[['professional sector', 'company']] = df1.Source.str.split(' ==> ', expand=True)
#df1.drop(['Source'], axis='columns', inplace=True)

    #Reset index
    
df1 = df1.reset_index(drop=True)
df1.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/df1_dbclean.csv', index=False)
display(df1)
print('OK')

,Source,worth BUSD,worthChange millions USD,age,gender,country,image,name,position
0,technology ==> amazon,112,0,54,male,none,https://specials-images.forbesimg.com/imageser...,jeff bezos,1
1,technology ==> microsoft,90,0,62,none,united states,https://specials-images.forbesimg.com/imageser...,bill gates,2
2,finance and investments ==> berkshire hathaway,84,0,87,male,united states,https://specials-images.forbesimg.com/imageser...,warren buffett,3
3,fashion & retail ==> lvmh,72,0,69,none,none,https://specials-images.forbesimg.com/imageser...,bernard arnault,4
4,technology ==> facebook,71,0,35,male,none,https://specials-images.forbesimg.com/imageser...,mark zuckerberg,5
...,...,...,...,...,...,...,...,...,...
2203,"fashion & retail ==> fashion, entertainment",1,0,51,none,china,https://specials-images.forbesimg.com/imageser...,zhao xiaoqiang,2204
2204,manufacturing ==> electrical equipment,1,0,55,male,none,https://specials-images.forbesimg.com/imageser...,zhou liangzhang,2205
2205,manufacturing ==> electrical equipment,1,0,51,none,china,https://specials-images.forbesimg.com/imageser...,zhu xingming,2206
2206,manufacturing ==> printed circuit boards,1,0,52,female,none,https://specials-images.forbesimg.com/imageser...,zhuo jun,2207


OK


In [5]:
#WEB SCRAPPING:
    #To expand the database I used a wikipedia page, data obtained through WebScrapping:
        #Extract URL - Scrapper:

url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

table = soup.find_all('table',{'class':'wikitable sortable mw-datatable'})[0]
table

        # tr represent the table rows:
rows = table.find_all('tr')
rows_parsed = [row.text for row in rows]
rows_parsed

        #Parsering:

def smart_parser(row_text):
    row_text = row_text.replace('\nPopulation\n\n\n', '\nPopulation\n').strip('\n')
    row_text = row_text.replace('\n\n', '\n').strip('\n')
    row_text = row_text.replace('\nPopulation\n% of World\n', '\nPopulation % of World\n').strip('\n')
    row_text = re.sub('\[\d\]', '', row_text)
    
    return list(map(lambda x: x.strip(), row_text.split('\n')))

well_parsed = list(map(lambda x: smart_parser(x), rows_parsed))

colnames = well_parsed[0]
data = well_parsed[1:]

        #Creating DF:
df_scrapping = pd.DataFrame(data, columns=colnames)

df_scrapping['Country(or dependent territory)']=df_scrapping['Country(or dependent territory)'].replace(['\[(.*?)\]','\(([^\)]+)\)'], ['',''], regex=True)

df_scrapping.columns = ['Rank', 'country','Population',' % of World Population','Date','Source']

        #Exporting to CSV: 

df_scrapping.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/df_scrapping.csv', index=False)
display(df_scrapping)
print('OK.2')

,Rank,country,Population,% of World Population,Date,Source
0,1,China,"1,400,825,000",18.1%,10 Jan 2020,National population clock
1,2,India,"1,357,165,850",17.5%,10 Jan 2020,National population clock
2,3,United States,"330,563,468",4.26%,10 Jan 2020,National population clock
3,4,Indonesia,"266,911,900",3.44%,1 Jul 2019,National annual projection
4,5,Pakistan,"218,231,000",2.81%,10 Jan 2020,National population clock
...,...,...,...,...,...,...
236,–,Tokelau,"1,400",0.0000180%,1 Jul 2018,National annual estimate[94]
237,–,Vatican City,799,0.0000103%,1 Jul 2019,UN projection
238,–,Cocos Islands,538,0.00000693%,30 Jun 2018,National estimate[197]
239,–,Pitcairn Islands,50,0.000000644%,1 Jan 2019,National estimate[198]


OK.2


In [6]:
#Replacement of 'None' values by NaN in country column

df1.country.replace('None', np.nan, regex=True, inplace=True)

#Drop Null values in df_total - column: country
df1.country.isnull().sum()
df_no_null_values = df1[df1.age.isnull()]
df_no_null_values.shape


df1.drop(df1[df1.country.isnull()].index, inplace = True)
df1.shape
df1.country.isnull().sum()

#Merge - df_total,no null values in country col with df_web_scrapping:

df_try1 = pd.merge(df_scrapping, df1, on=['country', 'country'])

#Exportin to processed:

df_try1.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/df_try1.csv', index=False)
df_try1

,Rank,country,Population,% of World Population,Date,Source_x,Source_y,worth BUSD,worthChange millions USD,age,gender,image,name,position
